In [1]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, sqlite3, pickle, time, datetime, random

from sklearn.model_selection import LeaveOneOut
import os

t = time.time()

# 0. Load Data a Parameters

In [2]:
#######################################
### Relevant fields for pre-processing
#######################################
reference_date = datetime.datetime(1970,1,1)

##### Fields in table patients ##########
patient_field         = 'patient_id'
age_field             = 'age_at_admission\r'
sex_field             = 'sex'
ethnic_field          = 'ethnic_origin'
death_ind_field       = 'death_indicator'
death_date_field      = 'date_of_death'
death_date_code_field = 'date_code_of_death'
mort_in_hosp_field    = 'Thirty_day_mort'

##### Fields in table admissions ########
admission_field           = 'admission_id'
diagnosis_field           = 'episode_diagnoses'
admn_date_field           = 'admission_date_time'
admn_discharge_field      = 'discharge_date_time'
admn_date_code_field      = 'admission_date_code_time'
admn_discharge_code_field = 'discharge_date_code_time'
lengthofstay_field        = 'lengthofstay'
isPneumonia_field         = 'isPneumonia'
mortal_admin_field        = 'mortal_admin'
comorbidity_field         = 'Comorbidity_score'
icu_admin_field           = 'icu_count\r'
no_eobs_field             = 'no_obs_eobs'
no_haematology_field      = 'no_haematology_eobs'

 ##### Fields in table eObservations #####
eObs_time_field      = 'timestamp'
eObs_time_code_field = 'timestamp_code'
eObs_time_prev_obs   = 'time_since_prev_obs_in_mins'
eObs_resprate_field  = 'rr'
eObs_sbp_field       = 'sbp'
eObs_dbp_field       = 'dbp'
eObs_newscore_field  = 'ews'
eObs_heartrate_field = 'heart_rate'
eObs_temptr_field    = 'temperature\r'
eObs_oxygen_field    = 'Oxygen_Saturation'

 ##### Fields in table haematological tests #####
test_code_field      = 'local_test_code'
test_time_field      = 'sample_collection_date_time'
test_time_code_field = 'sample_collection_date_code_time'

### 0.1. Load Data

In [3]:
##############################################################################
# 0. LOAD DATA AND PARAMETERS
##############################################################################
data = pickle.load( open('DataFrame_pickles/df_patients_admissions_v3.pickle', 'rb'))
df_patients   = data[0]
df_admissions = data[1]
data = pickle.load( open('DataFrame_pickles/df_haematology_v1.pickle', 'rb'))
df_haematology  = data[0]

list_admissions      = df_admissions[admission_field].unique().tolist()
list_haemat_features = list(df_haematology.columns[2:])

# 1. Definition of Function for interpolation

In [4]:
##############################################################################
# 1. DEFINITION OF FUNCTIONS
##############################################################################
# 1.1. interpolate_by_method ===========================================
def interpolate_by_method(series, method, plot = 0):
    if ('polynomial' in method) or ('nearest' in method):
        interpolate = series.interpolate(method = method[:-1], order = int(method[-1]))
    else:
        interpolate = series.interpolate(method = method, order = method)
    if plot != 0: interpolate.plot()
    return interpolate
# 1.2. cross_validation_series ===========================================
def cross_validation_series(feature_series, times, interp_methods, upsamptimes):    
    cv     = LeaveOneOut()
    errors = np.zeros([len(feature_series),len(interp_methods)])
    h = 0
    for train_ix, test_ix in cv.split(times):
        
        feat_train, time_train = feature_series.iloc[train_ix], times.iloc[train_ix]
        feat_test, time_test   = feature_series.iloc[test_ix], times.iloc[test_ix]
             
        values      = [feat_train[time_train == tm].mean() for i,tm in enumerate(upsamptimes)]
        cv_series   = pd.Series(values, index=upsamptimes)
        
        for idx_method, method in enumerate(interp_methods):
            if (cv_series.count() < 4  and method == 'polynomial3') or (method == 'polynomial2' and cv_series.count() < 3): 
                errors[h, idx_method] = 1
                continue
            interp = interpolate_by_method(cv_series, method)
            errors[h, idx_method] = (interp.loc[time_test].values[0] - feat_test.values[0]) ** 2
        h = h + 1    
        
    errors  = np.nanmean(errors, axis=0)
    errors  = np.sqrt(errors) # RMSE
    error   = errors.min()
    methods = [interp_methods[i] for i, x in enumerate(errors == np.nanmin(errors)) if x]
    return methods[0], errors

# 2. Construction of df_old 
### Counts the number of samples of each admission and count the bt available in each admission. 
### to get the number of admissions to interpolate

In [5]:
t2 = time.time()
#list_haematology_tests = df_haematology.columns[3:].tolist()
list_admissions_haemt  = df_haematology[admission_field].unique().tolist()
df_old = []
for admin in list_admissions_haemt:
    row = [admin]
    temp = df_haematology[(df_haematology[admission_field] == admin)]
    for test in list_haemat_features:
        row.append(temp[test].notnull().sum())
    row.append(len(temp))
    df_old.append(row)
df_old = pd.DataFrame(df_old, columns = [admission_field] + list_haemat_features + ['totals'])
display(df_old.head(5))
print('Elapsed in extracting haematology information:', time.time()-t2)

,admission_id,CREA,UREA,K,EGFR,GFR,WBC,PLT,HCT,HGB,...,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI,totals
0,9297156,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,91590293,15,15,15,15,15,14,14,14,14,...,14,14,14,14,14,14,3,3,3,15
2,43790054,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,5,5,5,8
3,13019930,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,2,2,1,4
4,51403915,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,2,2,2,3


Elapsed in extracting haematology information: 53.84029960632324


In [6]:
##############################################################################
# LIST OF ADMISSIONS TO INTERPOLATE
# Depending on length of stay and have more than threshold cut observations
##############################################################################
los_cut_threshold = 1 # Threshold in days of length of stay
obs_cut_threshold = 2 # Threshold of number of observations per admissions
admissions_los_thr = df_admissions[df_admissions[lengthofstay_field] >= datetime.timedelta(days=los_cut_threshold)][admission_field]
admissions_m1t_obs = df_old[df_old['totals'] >  obs_cut_threshold][admission_field]
admissions_interp  = list(set(admissions_los_thr) & set(admissions_m1t_obs))

print("number of admissions to interpolate", len(admissions_interp))

number of admissions to interpolate 9704


# 3. Interpolation of Haematology

In [7]:
t = time.time()
new_haematology_3d = pd.DataFrame(columns = df_haematology.columns)
new_haematology_2d = pd.DataFrame(columns = df_haematology.columns)
new_haematology_1d = pd.DataFrame(columns = df_haematology.columns)
admins_pass_1 = []
admins_pass_2 = []
for idx, adm in enumerate(admissions_interp):
    
    t1 = time.time()
    
    adm_los  = df_admissions[df_admissions[admission_field] == adm].iloc[0][lengthofstay_field]
    adm_haem = df_haematology[df_haematology[admission_field]==adm].copy()    
    #list_haem_feat = adm_haem.columns[3:]
    times          = adm_haem[test_time_field]
    
    times_3d = times[times<= (times.min() + datetime.timedelta(days=3))]
    times_3d = times.iloc[:len(times_3d) + 1]
    
    if (adm_los.days < 1):
        admins_pass_1.append(adm)
    elif (len(times_3d) < 3) : 
        admins_pass_2.append(adm)
        continue
    #round to the closest 6th of hour
    times_3d = times_3d.apply(lambda dt: datetime.datetime(dt.year, dt.month, 
                                                       dt.day, 
                                                       min(round(dt.hour/6)*6,18),
                                                       0))
    upsamp_times    = pd.date_range(times_3d.min(), times_3d.max(), freq = '6H').tolist()
    upsamp_features = np.zeros([len(upsamp_times),len(list_haemat_features)])
    
    for idx_feat, feature in enumerate(list_haemat_features):
        #if feature == 'ALB':break
        feature_series = adm_haem[feature].iloc[:len(times_3d)]    
        values  = [feature_series[times_3d == tm].mean() for i,tm in enumerate(upsamp_times)]        
        series_ = pd.Series(values, index=upsamp_times)
        
        if series_.notnull().sum() == 0: # If all of them are missing skip feature
            upsamp_features[:, idx_feat] = np.nan
            continue            
        elif series_.notnull().sum() == 1: # If there is only one value leave all the same value
            upsamp_features[:, idx_feat] = series_[series_.notnull()].iloc[0]
            continue
        
        interp  = interpolate_by_method(series_, 'linear') #'pchip')
        if interp.isnull().sum() > 0: interp = interp.interpolate(method='backfill', limit = 20, limit_direction = 'backward')            
        upsamp_features[:, idx_feat] = interp
        
    df = pd.DataFrame(upsamp_features, columns = list_haemat_features)
    df[admission_field] = adm   
    df[test_time_field] = upsamp_times
    
    if (adm_los.days >= 3) and (len(df) >= 4*3): new_haematology_3d  = pd.concat([new_haematology_3d, df.iloc[:4*3]])
    if (adm_los.days >= 2) and (len(df) >= 4*2): new_haematology_2d  = pd.concat([new_haematology_2d, df.iloc[:4*2]])
    if (adm_los.days >= 1) and (len(df) >= 4*1): new_haematology_1d  = pd.concat([new_haematology_1d, df.iloc[:4*1]])
    print(idx, 'Admission: ',adm, ' is ', time.time() - t1)
print('Elapsed in extracting haematology information:', time.time()-t)

0 Admission:  1671170  is  0.12523174285888672
1 Admission:  7077893  is  0.13865256309509277
2 Admission:  29491206  is  0.24552130699157715
3 Admission:  35553286  is  0.14307188987731934
4 Admission:  68616199  is  0.19123387336730957
5 Admission:  51445770  is  0.15912866592407227
7 Admission:  55148554  is  0.1639387607574463
8 Admission:  56492045  is  0.15077805519104004
9 Admission:  55083022  is  0.17013144493103027
10 Admission:  73859084  is  0.14480257034301758
11 Admission:  85393425  is  0.16355228424072266
12 Admission:  34177047  is  0.1903841495513916
13 Admission:  36569112  is  0.26410698890686035
14 Admission:  93192217  is  0.13610410690307617
15 Admission:  95780890  is  0.1197519302368164
16 Admission:  36372509  is  0.1661825180053711
17 Admission:  12550178  is  0.1715221405029297
18 Admission:  49315879  is  0.1391606330871582
19 Admission:  44367913  is  0.15515494346618652
21 Admission:  52297773  is  0.14090180397033691
23 Admission:  10518575  is  0.188980

189 Admission:  99648078  is  0.08453249931335449
190 Admission:  76513878  is  0.11483502388000488
191 Admission:  55935579  is  0.11750960350036621
192 Admission:  42828382  is  0.14925861358642578
194 Admission:  52822626  is  0.13277983665466309
196 Admission:  54428272  is  0.1700150966644287
197 Admission:  44204658  is  0.173750638961792
199 Admission:  99353219  is  0.16283464431762695
200 Admission:  23593604  is  0.14356708526611328
201 Admission:  27918983  is  0.20415568351745605
202 Admission:  25559699  is  0.13241863250732422
203 Admission:  37814937  is  0.11431694030761719
204 Admission:  10027673  is  0.12498807907104492
205 Admission:  41550489  is  0.1524827480316162
207 Admission:  35979943  is  0.17582464218139648
209 Admission:  86311596  is  0.21821928024291992
210 Admission:  44499629  is  0.13563013076782227
211 Admission:  66650796  is  0.1342334747314453
212 Admission:  89129652  is  0.11466097831726074
214 Admission:  1376950  is  0.13780736923217773
215 Ad

384 Admission:  41321765  is  0.14822077751159668
386 Admission:  40535340  is  0.13460612297058105
387 Admission:  24315181  is  0.17340970039367676
388 Admission:  74908975  is  0.13468170166015625
389 Admission:  78251315  is  0.13799309730529785
390 Admission:  57312563  is  0.16217446327209473
391 Admission:  84051260  is  0.16495418548583984
392 Admission:  52102461  is  0.11583089828491211
393 Admission:  98567486  is  0.16118717193603516
394 Admission:  91751746  is  0.1903367042541504
396 Admission:  94373199  is  0.13486552238464355
397 Admission:  80840020  is  0.15178370475769043
399 Admission:  5408090  is  0.1123504638671875
400 Admission:  79791451  is  0.13441205024719238
401 Admission:  65668453  is  0.13555121421813965
402 Admission:  26412390  is  0.16398143768310547
403 Admission:  86017383  is  0.12692809104919434
404 Admission:  15205736  is  0.1047828197479248
405 Admission:  68093292  is  0.15338850021362305
406 Admission:  74417517  is  0.15416765213012695
407 

572 Admission:  48891751  is  0.1374835968017578
575 Admission:  94930804  is  0.1335303783416748
576 Admission:  37914485  is  0.13846969604492188
577 Admission:  34932598  is  0.11766648292541504
578 Admission:  29460347  is  0.16715335845947266
579 Admission:  3803008  is  0.12941527366638184
580 Admission:  31655810  is  0.12767982482910156
581 Admission:  657283  is  0.12990260124206543
582 Admission:  37783429  is  0.14879202842712402
583 Admission:  40961926  is  0.12963151931762695
584 Admission:  14616455  is  0.17541027069091797
585 Admission:  15665033  is  0.15390348434448242
586 Admission:  57477002  is  0.13412928581237793
588 Admission:  41060238  is  0.1661059856414795
589 Admission:  79529879  is  0.11308574676513672
590 Admission:  60524440  is  0.12884736061096191
591 Admission:  52627366  is  0.12132573127746582
592 Admission:  37914537  is  0.13015985488891602
593 Admission:  48564139  is  0.14707684516906738
594 Admission:  73238445  is  0.17577767372131348
595 Ad

760 Admission:  10848629  is  0.17644238471984863
761 Admission:  77171068  is  0.09680342674255371
762 Admission:  84314495  is  0.2184152603149414
763 Admission:  98699648  is  0.14061379432678223
764 Admission:  30181763  is  0.14348769187927246
765 Admission:  89819525  is  0.16426396369934082
766 Admission:  35621255  is  0.20634865760803223
767 Admission:  28412306  is  0.13340067863464355
768 Admission:  47516050  is  0.17889642715454102
770 Admission:  98765209  is  0.14113593101501465
771 Admission:  51186075  is  0.13331007957458496
772 Admission:  67275174  is  0.12479758262634277
773 Admission:  28838311  is  0.1216585636138916
774 Admission:  95521192  is  0.14185047149658203
775 Admission:  89852328  is  0.12379622459411621
776 Admission:  30673329  is  0.12659811973571777
777 Admission:  16157108  is  0.17005205154418945
778 Admission:  89655734  is  0.13044333457946777
779 Admission:  77072823  is  0.13053131103515625
780 Admission:  22383037  is  0.1342296600341797
781

...

8829 Admission:  91354197  is  0.21892166137695312
8830 Admission:  48460879  is  0.222367525100708
8831 Admission:  15725656  is  0.3455617427825928
8832 Admission:  62714971  is  0.2142648696899414
8833 Admission:  31945822  is  0.13552284240722656
8834 Admission:  24638559  is  0.218339204788208
8835 Admission:  83653732  is  0.21987056732177734
8836 Admission:  38401136  is  0.2233893871307373
8837 Admission:  28406896  is  0.22466516494750977
8838 Admission:  45937790  is  0.23223495483398438
8839 Admission:  97121407  is  0.23376250267028809
8840 Admission:  88175744  is  0.22732901573181152
8842 Admission:  31159434  is  0.29386401176452637
8843 Admission:  63730830  is  0.25609922409057617
8844 Admission:  41448594  is  0.27103710174560547
8845 Admission:  30143634  is  0.23207426071166992
8846 Admission:  78771348  is  0.2848086357116699
8847 Admission:  60454039  is  0.23317956924438477
8848 Admission:  30864543  is  0.23430204391479492
8849 Admission:  34567328  is  0.302809

9013 Admission:  27195093  is  1.3180136680603027
9014 Admission:  38008535  is  0.3262171745300293
9015 Admission:  75495126  is  0.25897884368896484
9016 Admission:  11597530  is  0.2278139591217041
9017 Admission:  45938394  is  0.2623598575592041
9018 Admission:  19822304  is  0.22763633728027344
9019 Admission:  71038691  is  0.2251574993133545
9021 Admission:  79591141  is  0.2526097297668457
9022 Admission:  44725991  is  0.2568626403808594
9023 Admission:  82540272  is  0.2232367992401123
9024 Admission:  6878963  is  0.22319984436035156
9025 Admission:  33257204  is  0.0965585708618164
9026 Admission:  22705915  is  0.25818705558776855
9027 Admission:  87193342  is  0.19844555854797363
9028 Admission:  34633473  is  0.22929787635803223
9029 Admission:  1275652  is  0.24966192245483398
9030 Admission:  11597572  is  0.2312459945678711
9031 Admission:  67204870  is  0.23825931549072266
9032 Admission:  30832414  is  0.22212624549865723
9033 Admission:  5437219  is  0.26988601684

9212 Admission:  33814920  is  0.2240447998046875
9213 Admission:  58194314  is  0.23351311683654785
9214 Admission:  13662603  is  0.22933721542358398
9215 Admission:  56129936  is  0.2489311695098877
9216 Admission:  58128790  is  0.2977182865142822
9217 Admission:  36370840  is  0.23132586479187012
9219 Admission:  63011235  is  0.21962642669677734
9220 Admission:  9763237  is  0.3072504997253418
9221 Admission:  2783654  is  0.2619314193725586
9222 Admission:  92764587  is  0.2647716999053955
9224 Admission:  15104427  is  0.2776145935058594
9225 Admission:  72677811  is  0.2383708953857422
9226 Admission:  85195189  is  0.23943161964416504
9227 Admission:  26540470  is  0.26221227645874023
9228 Admission:  11762110  is  0.25363969802856445
9229 Admission:  63568322  is  0.22507500648498535
9230 Admission:  19823047  is  0.258270263671875
9231 Admission:  62683592  is  0.26421499252319336
9232 Admission:  44792265  is  0.24083375930786133
9233 Admission:  85948877  is  0.2323539257

9393 Admission:  1276932  is  0.23068737983703613
9395 Admission:  34274311  is  0.2330784797668457
9396 Admission:  56556552  is  0.235093355178833
9397 Admission:  93060102  is  0.23136472702026367
9398 Admission:  14154768  is  0.24079108238220215
9400 Admission:  38599699  is  0.15758180618286133
9401 Admission:  26704917  is  0.24390769004821777
9402 Admission:  96205847  is  0.24017000198364258
9403 Admission:  87391258  is  0.14567255973815918
9405 Admission:  31849505  is  0.2511007785797119
9406 Admission:  41253926  is  0.24335074424743652
9407 Admission:  68254758  is  0.22697734832763672
9409 Admission:  26967082  is  0.24608111381530762
9410 Admission:  40565803  is  0.23273396492004395
9411 Admission:  12221483  is  0.22240972518920898
9412 Admission:  33291312  is  0.23503518104553223
9413 Admission:  42925107  is  0.24864697456359863
9414 Admission:  93551677  is  0.23891639709472656
9415 Admission:  21593151  is  0.2169187068939209
9417 Admission:  49740864  is  0.2731

9578 Admission:  95256107  is  0.24073266983032227
9579 Admission:  31391278  is  0.24265599250793457
9580 Admission:  44432950  is  0.23269987106323242
9581 Admission:  65535546  is  0.228898286819458
9582 Admission:  52592186  is  0.251781702041626
9583 Admission:  78544450  is  0.30173420906066895
9584 Admission:  84606532  is  0.2509267330169678
9585 Admission:  23789125  is  0.23485970497131348
9586 Admission:  5897800  is  0.3191981315612793
9587 Admission:  917066  is  0.246596097946167
9588 Admission:  86769231  is  0.2568187713623047
9591 Admission:  26934874  is  0.25693583488464355
9592 Admission:  30735962  is  0.24028801918029785
9593 Admission:  57474652  is  0.2240591049194336
9594 Admission:  99188321  is  0.2578742504119873
9595 Admission:  80707176  is  0.23644351959228516
9596 Admission:  13467245  is  0.23069024085998535
9597 Admission:  96927348  is  0.23264455795288086
9598 Admission:  88243837  is  0.25247645378112793
9599 Admission:  15007360  is  0.302296638488

In [8]:
print("Number of admissions not processed that were not well filtered by LoS")
print(len(admins_pass_1))
print("Number of admissions with less than 2 observations to be interpolated:")
print(len(admins_pass_2))

Number of admissions not processed that were not well filtered by LoS
0
Number of admissions with less than 2 observations to be interpolated:
1271


In [9]:
print("len 1d:", len(new_haematology_1d['admission_id'].unique()))
print("len 2d:", len(new_haematology_2d['admission_id'].unique()))
print("len 3d:", len(new_haematology_3d['admission_id'].unique()))

len 1d: 8430
len 2d: 8399
len 3d: 8258


In [10]:
new_haematology_3d.head(4)

,admission_id,sample_collection_date_time,CREA,UREA,K,EGFR,GFR,WBC,PLT,HCT,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
0,1671170,2020-03-31 00:00:00,59.0,15.5,4.4,90.0015,90.0015,11.3,293.0,0.414,...,29.4,91.0,7.56,2.5,0.23,0.96,0.06,41.0,112.0,4.0
1,1671170,2020-03-31 06:00:00,59.2,15.36,4.42,90.0015,90.0015,11.4,290.6,0.4164,...,29.38,91.0,7.678,2.466,0.238,0.958,0.058,40.4,112.2,4.2
2,1671170,2020-03-31 12:00:00,59.4,15.22,4.44,90.0015,90.0015,11.5,288.2,0.4188,...,29.36,91.0,7.796,2.432,0.246,0.956,0.056,39.8,112.4,4.4
3,1671170,2020-03-31 18:00:00,59.6,15.08,4.46,90.0015,90.0015,11.6,285.8,0.4212,...,29.34,91.0,7.914,2.398,0.254,0.954,0.054,39.2,112.6,4.6


In [11]:
df = pd.concat([new_haematology_3d.isna().sum(), new_haematology_3d.dtypes], axis=1)
df = df.rename(columns ={0:'nulls',1:'type'})
display(df.sort_values(by=['nulls']))

,nulls,type
admission_id,0,object
WBC,0,object
K,0,object
HGB,0,object
CREA,0,object
UREA,0,object
sample_collection_date_time,0,object
MCV,12,object
RBC,12,object
MCH,12,object


In [12]:
pickle.dump(new_haematology_3d, open( "DataFrame_pickles/new_haematology_3d.pickle", "wb" ))